In [22]:
# Task 2: prepare two-qubit arbitrary complex amplitudes without using `initialize` helpers
import pennylane as qml
from pennylane import numpy as np

In [29]:
# --- utilities ---
def normalize_amplitudes(a):
    a = np.array(a, dtype=complex).reshape(-1)
    norm = np.sqrt(np.sum(np.abs(a) ** 2))
    if norm == 0:
        raise ValueError("Zero vector provided")
    return a / norm

In [30]:
def vector_to_rotations(u):
    """Return (theta, phi) such that
       [cos(theta/2), exp(i*phi) sin(theta/2)] ≈ u (up to global phase)."""
    u = np.array(u, dtype=complex).reshape(2)
    norm = np.linalg.norm(u)
    if norm == 0:
        return 0.0, 0.0
    u = u / norm
    theta = 2 * np.arccos(np.clip(np.abs(u[0]), -1.0, 1.0))
    phi = 0.0
    if np.abs(u[1]) > 1e-12:
        phi = np.angle(u[1]) - np.angle(u[0])
    return float(theta), float(phi)

In [31]:
# --- core prep routine ---
def prepare_two_qubit_state(amplitudes):
    """Decompose arbitrary [a0,a1,a2,a3] into rotations/controls on 2 qubits."""
    a = normalize_amplitudes(amplitudes)
    r0 = np.linalg.norm(a[0:2])
    r1 = np.linalg.norm(a[2:4])

    # Step 1: put qubit 0 into r0|0> + r1|1>
    theta0 = 2 * np.arctan2(r1, r0)
    qml.RY(theta0, wires=0)

    # Step 2: conditionally prepare qubit 1
    if r0 > 1e-12:
        v0 = a[0:2] / r0
        th, ph = vector_to_rotations(v0)
        qml.ctrl(qml.RY, control=0)(th, wires=1)
        qml.ctrl(qml.RZ, control=0)(ph, wires=1)
    if r1 > 1e-12:
        v1 = a[2:4] / r1
        th, ph = vector_to_rotations(v1)
        qml.PauliX(0)
        qml.ctrl(qml.RY, control=0)(th, wires=1)
        qml.ctrl(qml.RZ, control=0)(ph, wires=1)
        qml.PauliX(0)

In [32]:
# --- device and circuit ---
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(amplitudes):
    prepare_two_qubit_state(amplitudes)
    return qml.state()

In [33]:
# --- test ---
if __name__ == "__main__":
    amps = [0.5, 0.5j, 0.5, -0.5j]  # arbitrary
    out = circuit(amps)
    target = normalize_amplitudes(amps)
    fid = np.abs(np.vdot(target, out))**2
    print("Target:", target)
    print("Prepared:", out)
    print("Fidelity:", fid)

Target: [ 0.5+0.j   0. +0.5j  0.5+0.j  -0. -0.5j]
Prepared: [0.35355339+0.35355339j 0.35355339-0.35355339j 0.35355339-0.35355339j
 0.35355339+0.35355339j]
Fidelity: 0.0
